### Customers

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
import os, sys

repo_root = os.getcwd() 
if repo_root not in sys.path:
    sys.path.append(repo_root)

In [0]:
df_cust = spark.read.table("pysparkdbt.bronze.customers")

In [0]:
display(df_cust)

In [0]:
df_cust = df_cust.withColumn("domain", split(col("email"), "@").getItem(1))
display(df_cust)

In [0]:
# Replace all non digits letters in the phone number column with nothing.
df_cust = df_cust.withColumn(
    "phone_number",
    regexp_replace(col("phone_number"), "[^0-9]", "")
)
display(df_cust)

In [0]:
# Created full_name column by concatinating first_name and last_name, dropping the original after.
df_cust = df_cust.withColumn("full_name", concat_ws((" "), col("first_name"), col("last_name")))
display(df_cust)
df_cust = df_cust.drop("first_name", "last_name")

In [0]:
from utilities.custom_utils import Transformations
t = Transformations()
df_transformed = t.deduplicate(df_cust,["customer_id"],"last_updated_timestamp", "ingestion_timestamp")

display(df_transformed)

In [0]:
df_transformed = t.process_timestamp(df_transformed)
display(df_transformed)

In [0]:
from pyspark.sql.functions import col

table = "pysparkdbt.silver.customers"
cdc_col = "last_updated_timestamp"
t.upsert(
        df=df_transformed,
        key_cols=["customer_id"],
        target_table=table,
        cdc=cdc_col
    )

In [0]:
display(df_transformed) # Rerun and should constantly be 200 if no new records have been inserted.

### Drivers Table

In [0]:
df_driver = spark.read.table("pysparkdbt.bronze.drivers")
display(df_driver)

In [0]:
df_driver = df_driver.withColumn("full_name", concat_ws((" "), col("first_name"), col("last_name")))
display(df_driver)
df_driver = df_driver.drop("first_name", "last_name")

In [0]:
# Replace all non digits letters in the phone number column with nothing.
df_driver = df_driver.withColumn(
    "phone_number",
    regexp_replace(col("phone_number"), "[^0-9]", "")
)
display(df_driver)

In [0]:
# Checking for duplicates

dup_primary = t.duplicates_report(df_driver, ["driver_id"])
dup_temporal = t.duplicates_report(df_driver, ["driver_id", "last_updated_timestamp"])
display(dup_primary)
display(dup_temporal)

In [0]:
nulls = t.nulls_report(df_driver, ["driver_id", "last_updated_timestamp", "phone_number", "vehicle_id", "driver_rating", "city"	])
display(nulls)

In [0]:
# Split the data into valid and rejected by assuring non nulls
split = t.require_non_null(df_driver, ["driver_id" , "last_updated_timestamp"])
valid = split["valid"]
rejected = split["rejected"]

In [0]:
# Filters by valid ranges on the driver ratings [0.0,5.0]
valid = t.filter_by_ranges(valid, {"driver_rating": (0.0, 5.0)}, inclusive=True)["valid"]

In [0]:
# Deduplication by last updated timestamp and if needed ingestion timestamp as a tie breaker
df_silver = t.deduplicate_by_recency(
    df = valid,
    keys = ["driver_id"],
    cdc = "last_updated_timestamp",
    tie_breaker = "ingestion_timestamp"
)

In [0]:
display(df_silver)

In [0]:
t.upsert(
    df = df_silver,
    key_cols = ["driver_id"],
    target_table = "pysparkdbt.silver.drivers",
    cdc = "last_updated_timestamp"
)

### Vehicles

In [0]:
df_vehicles = spark.read.table("pysparkdbt.bronze.vehicles")
display(df_vehicles)

In [0]:
# Clean Strings
import pyspark.sql.functions as F
df_vehicles = t.sanitize_string(df_vehicles, ["license_plate"], pattern=r"[^A-Za-z0-9]", replacement="")
df_vehicles = df_vehicles.withColumn("license_plate", F.upper(F.col("license_plate")))
display(df_vehicles)


In [0]:
df_vehicles = t.sanitize_string(df_vehicles, ["make"], pattern=r",", replacement=" ")
df_vehicles = t.sanitize_string(df_vehicles, ["make"], pattern=r"[^A-Za-z0-9 .&'\-]", replacement="")
df_vehicles = df_vehicles.withColumn("make", F.regexp_replace(F.col("make"), r"\s+", " "))
df_vehicles = df_vehicles.withColumn("make", F.initcap(F.trim(F.col("make"))))
display(df_vehicles)

In [0]:
df_vehicles = t.sanitize_string(df_vehicles, ["model"], pattern=r"[^A-Za-z0-9 \-]", replacement="")
df_vehicles = df_vehicles.withColumn("model", F.regexp_replace(F.col("model"), r"\s+", " "))
df_vehicles = df_vehicles.withColumn("model", F.trim(F.col("model")))
display(df_vehicles)

In [0]:
%sql
SELECT DISTINCT vehicle_type FROM pysparkdbt.bronze.vehicles

In [0]:
valid_types = F.array([F.lit(x) for x in ["Sedan","Suv","Hatchback","Van","Luxury"]])
df_vehicles = df_vehicles.withColumn("vehicle_type", F.initcap(F.trim(F.col("vehicle_type"))))

display(df_vehicles)

In [0]:
from datetime import datetime
max_year = datetime.now().year
res_year = t.filter_by_ranges(df_vehicles, {"year": (1980.0, float(max_year))}, inclusive=True)
df_vehicles = res_year["valid"]
df_vrejected = res_year["rejected"]
display(df_vrejected)
display(df_vehicles)

In [0]:
duplicated = t.duplicates_report(df_vehicles, ["license_plate", "vehicle_id"])
display(duplicated)

nulls = t.nulls_report(df_vehicles, ["vehicle_id","license_plate","year","vehicle_type"])
display(nulls)

In [0]:
split = t.require_non_null(df_vehicles, ["vehicle_id"])
valid = split["valid"]
rejected = split["rejected"]

display(valid)
display(rejected)

In [0]:
# Deduplication on the vehicle id key by recency used last updated timestmamp or the ingestion timestamp
df_silver = t.deduplicate_by_recency(
    df=valid,
    keys=["vehicle_id"],
    cdc="last_updated_timestamp",
    tie_breaker="ingestion_timestamp"
) if "last_updated_timestamp" in valid.columns else t.deduplicate_by_recency(valid, ["vehicle_id"], cdc="vehicle_id")

In [0]:
t.upsert(
    df=df_silver,
    key_cols=["vehicle_id"],
    target_table="pysparkdbt.silver.vehicles",
    cdc="last_updated_timestamp" if "last_updated_timestamp" in df_silver.columns else "vehicle_id"
)

In [0]:
%sql
SELECT COUNT(*) FROM pysparkdbt.silver.vehicles